In [1]:
from glob import glob
from tqdm import tqdm
import json
import re

In [2]:
rejected_words = [
    'kebutuhan',
    'berbeda',
    'bahwa',
    'Kode',
    'kode',
    'nomor',
    'RMXX,XXX',
    'kompleksitas',
    'listrik',
    'jawaban',
    'teknis',
    'berkualitas',
    'mencoba',
    'kampanye',
    'komunitas',
    'stabilitas',
    'Stabilitas',
    'metode',
    'pria',
    'butuh',
    'jadwal',
    'kasus',
    'otomatis',
    'populer',
    'bisnis',
    'probabilitas',
    'rusak',
    'kapasitas',
    'rutinitas'
]
rejected_words = set(rejected_words)

def check_indon(string):
    string = re.sub('[^A-Za-z ]+', ' ', string)
    return len(set(string.split()) & rejected_words) > 0

In [3]:
files = sorted(glob('evol-codealpaca-v1-chatgpt4/*.json'), key = lambda x: int(x.split('/')[-1].replace('.json', '')))
len(files)

41659

In [4]:
import tiktoken
enc = tiktoken.get_encoding("cl100k_base")

/home/husein/.local/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.2.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [5]:
rejected_instructions = [
    'tidak dapat melakukan penulisan semula', 
    'maaf', 
    'not able to fulfill your request',
    'saya tidak dapat membantu',
    'dengan permintaan itu',
    'feel free to ask',
    'saya tidak bisa',
]

In [6]:
count, indon_count = 0, 0

indon_texts = []
for f in tqdm(files):
    try:
        with open(f) as fopen:
            data = json.load(fopen)
    except:
        continue

    if data['output'] is None:
        continue
        
    if any([r in data['instruction'].lower() for r in rejected_instructions]):
        continue

    indon_texts.append(data['instruction'])
    indon_texts.append(data['output'])

    count += 1

100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 41659/41659 [00:00<00:00, 53085.44it/s]


In [7]:
len(indon_texts)

83026

In [8]:
print(indon_texts[1])

Untuk mengubah skrip ini, kita Akkan menggunakan konsep pengulangan 'while' dan memastikan index tidak melebihi panjang senarai. Juga, kita harus memastikan skrip kita menangani senarai kosong dengan betul. 

Berikut adalah cara mengubahnya:

```python
  # Menetapkan senarai integer
  arr = [1, 2, 3, 4]

  # Tentukan panjang senarai
  n = len(arr)

  # Pastikan senarai tidak kosong
  if n > 0:
    i = 0
    # Melalui senarai dan keluarkan setiap elemen individu
    while i < n:
        print(arr[i])
        i += 1
  else:
    print("Senarai adalah kosong.")
```

Dalam skrip ini, sebelum kita mulai mengulang, kita memeriksa apakah `n`, yang merupakan panjang senarai, lebih besar dari 0. Jika tidak (yang berarti senarai kosong), kita mencetak pesan yang sesuai. 

Kemudian, kami menggunakan pengulangan 'while' untuk mengulang selama indeks `i` kurang dari panjang `n` senarai. Selama setiap iterasi, kita mencetak elemen senarai dan tambahkan 1 ke indeks, sehingga kita dapat bergerak ke ele

In [9]:
with open('codealpaca-v1-chatgpt4.texts', 'w') as fopen:
    for t in set(indon_texts):
        fopen.write(f'{json.dumps(t)}\n')

In [11]:
len(set(indon_texts))

83007

In [13]:
!cp codealpaca-v1-chatgpt4.texts ~/ssd3/ctranslate2

In [45]:
mapping = {}
with open('/home/husein/ssd3/ctranslate2/code-instructions.texts.requested') as fopen:
    for l in fopen:
        l = json.loads(l)
        mapping[l['src']] = l['r']
        
len(mapping)

211739

In [10]:
rejected = 0

with open('synthetic-codealpaca-v1-chatgpt4.jsonl', 'w') as fopen_l:
    for f in tqdm(files):
        try:
            with open(f) as fopen:
                data = json.load(fopen)
        except:
            continue
            
        if data['output'] is None:
            continue
        
        
        indon_ins = False
        if check_indon(data['instruction']):
            indon_ins = True
        
        indon_output = False
        if check_indon(data['output']):
            indon_output = True
        
        rejected_ins = False
            
        if any([r in data['instruction'].lower() for r in rejected_instructions]):
            rejected_ins = True
        
        rejected_output = False
            
        if rejected_ins or rejected_output:
            rejected += 1
            
        data['indon_ins'] = indon_ins
        data['indon_output'] = indon_output
        data['rejected_ins'] = rejected_ins
        data['rejected_output'] = rejected_output
        
        fopen_l.write(f'{json.dumps(data)}\n')
        count += 1

100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 34822/34822 [00:03<00:00, 10038.21it/s]


In [ ]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj='ultrachat-glaive_coder_raw_text.jsonl',
    path_in_repo='ultrachat-glaive_coder_raw_text.jsonl',
    repo_id='mesolitica/chatgpt4-code-instruct',
    repo_type='dataset',
)